In [ ]:
import glob 
import tables
import corner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from scipy import stats
from astropy import units as u
from astropy.coordinates import SkyCoord

TSUN = 4.9169e-6
PC = 3.0856775807e16
CLIGHT = 299792458.

catalog_columns = ['f','colatitude','longitude','amplitude','inclination','polarization','phase','fdot','fddot','sigma_df/f','sigma_colatitude','sigma_longitude','sigma_dA/A','sigma_inclination','sigma_polarization','sigma_phase','sigma_dfdot/fdot','sigma_dfddot/fddot','sigma_Omega','SNR']
output_columns = catalog_columns + ['b','l','mchirp','distance']

In [ ]:
# # Import raw gbfisher data
# catalog = pd.read_table('SigmasAE.dat',delimiter=' ', header=None,index_col=False,names=catalog_columns)

# # Convert to galactic coordinates
# def ecliptic_to_galactic(row):
#     ecl_lon = row[2]*u.rad
#     ecl_lat = (np.pi/2 - row[1])*u.rad
#     gal = SkyCoord(ecl_lon,ecl_lat,frame='barycentrictrueecliptic').galactic
#     gal_lon = gal.l.degree
#     gal_lat = gal.b.degree
#     return pd.Series(dict(l=gal_lon,b=gal_lat))

# #catalog['b']=catalog.apply(lambda row: SkyCoord(catalog.longitude*u.rad,(np.pi/2 - catalog.colatitude)*u.rad,frame='barycentrictrueecliptic').galactic.b.degree, axis=1)
# catalog = catalog.join(catalog.apply(lambda row: ecliptic_to_galactic(row), axis=1))

# # Get chirp masses and distances (not sure how to get uncertainties on these)
# def amp_to_distance(row):
#     f = row[0]
#     A = row[3]
#     fdot = row[7]
#     Mc = 0
#     DL = 0
#     if(fdot>0):
#         Mc = (( fdot * (5./96.) / np.pi**(8./3.) / f**(11./3.))**(3./5.))/TSUN
#         DL = ((5./48.)*(fdot/(np.pi**2*f**3*A))*CLIGHT/PC)
#     return pd.Series(dict(mchirp=Mc,distance=DL))

# catalog = catalog.join(catalog.apply(lambda row: amp_to_distance(row), axis=1))

# # remove the binaries with unknown distance (and by extension unknown fdot and chirp mass too)   
# rem = catalog['distance'].between(0.0,0.001)
# catalog = catalog.drop(catalog.index[rem],axis=0)
# catalog.head()

# # Create gbfisher checkpoint file for speeding up data analysis
# gbfisher = pd.DataFrame(columns = output_columns)
# for col_name in output_columns:
#     gbfisher[col_name] = catalog[col_name]
# gbfisher.to_csv('gbfisher_checkpoint.dat',sep=' ',header=None,index=False)

In [ ]:
# Import gbfisher data
catalog = pd.read_table('gbfisher_checkpoint.dat',delimiter=' ', header=None,index_col=False,names=output_columns)
catalog.head()

In [ ]:
# SNRcut = catalog['SNR'].between(7,18)
# catalog = catalog[SNRcut]

In [ ]:
# Compute "luminosity"
catalog['luminosity'] = 1/(catalog['distance']**2)
catalog['luminosity'] = catalog['luminosity'] / sum(catalog['luminosity'])

# remove binaries with bad luminosity values (maybe look into this issue)
catalog = catalog.dropna(subset=['luminosity'])

In [ ]:
# Compute KDE
coords = np.vstack([catalog['l'],catalog['b']])
kernel = stats.gaussian_kde(coords)
kde = kernel.evaluate(coords).T / sum(kernel.evaluate(coords).T)

# take logarithm to weaken the influence of KDE over binary selection
catalog['KDE'] = pd.Series(-np.log(kde))
catalog = catalog.dropna(subset=['KDE'])

In [ ]:
# # Primitive weighting algorithm
# ckde = 1
# csnr = 2
# clum = 1

# catalog['weight'] = (catalog['KDE']**ckde)*(catalog['SNR']**csnr)*(catalog['luminosity']**clum)
# catalog['weight'] = catalog['weight'] / sum(catalog['weight'])
# catalog.head()

In [ ]:
ckde = 1
clum = 1

catalog['weight'] = (catalog['KDE']**ckde)*(catalog['luminosity']**clum)
catalog['weight'] = catalog['weight'] / sum(catalog['weight'])
catalog.head()

In [ ]:
# Apply cuts and plot result

weight_min = 3E-6  # 1.75E-4
weight_max = 1

Min_Period = 2/60
Max_Period = 20
f_min = 2/(60*Max_Period)
f_max = 2/(60*Min_Period)

sigma_incl_min = 0
sigma_incl_max = 10

incl_min = 89
incl_max = 90

snr_min = 0
snr_max = 20

cut = catalog['weight'].between(weight_min,weight_max) \
    & catalog['f'].between(f_min,f_max) \
    & np.degrees(catalog['sigma_inclination']).between(sigma_incl_min,sigma_incl_max) \
    & catalog['SNR'].between(snr_min,snr_max) \
    & np.degrees(catalog['inclination']).between(incl_min,incl_max)

f = plt.figure(figsize=(15,15))
plt.subplot(111, projection='aitoff')
plt.grid(True)
gal = SkyCoord(catalog[cut]['l'][:],catalog[cut]['b'][:],frame='galactic',unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=10)

catalog[cut].head(n=50)

In [ ]:
# # sky plot showing periods
# f = plt.figure(figsize=(15,15))
# plt.subplot(111, projection='aitoff')
# plt.grid(True)
# gal = SkyCoord(catalog[cut]['l'][:],catalog[cut]['b'][:],frame='galactic',unit=u.deg)
# plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=10,c=1/(30*catalog[cut]['f']),cmap='viridis')
# cbar = plt.colorbar(orientation='horizontal',pad=0.05)
# cbar.set_label(label='period',size=15)

# # sky plot showing distances
# f = plt.figure(figsize=(15,15))
# plt.subplot(111, projection='aitoff')
# plt.grid(True)
# gal = SkyCoord(catalog[cut]['l'][:],catalog[cut]['b'][:],frame='galactic',unit=u.deg)
# plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=10,c=catalog[cut]['distance'],cmap='viridis')
# cbar = plt.colorbar(orientation='horizontal',pad=0.05)
# cbar.set_label(label='distance',size=15)

# # sky plot showing inclinations
# f = plt.figure(figsize=(15,15))
# plt.subplot(111, projection='aitoff')
# plt.grid(True)
# gal = SkyCoord(catalog[cut]['l'][:],catalog[cut]['b'][:],frame='galactic',unit=u.deg)
# plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=10,c=90-abs(np.degrees(catalog[cut]['inclination'])-90),cmap='viridis')
# cbar = plt.colorbar(orientation='horizontal',pad=0.05)
# cbar.set_label(label='inclination',size=15)

# # sky plot showing SNRs
# f = plt.figure(figsize=(15,15))
# plt.subplot(111, projection='aitoff')
# plt.grid(True)
# gal = SkyCoord(catalog[cut]['l'][:],catalog[cut]['b'][:],frame='galactic',unit=u.deg)
# plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian,s=10,c=catalog[cut]['SNR'],cmap='viridis',norm=clr.LogNorm())
# cbar = plt.colorbar(orientation='horizontal',pad=0.05)
# cbar.set_label(label='SNR',size=15)

In [ ]:
# bins = 10**np.linspace(0.0, 2, 31)
# plt.hist(2/(60*catalog['f']), bins, histtype='bar', ec='black', color='C1', density=True)
# plt.title('Marginal Period Distribution for Population', fontsize=15)
# plt.xlabel('Period (minutes)',fontsize=14)
# #plt.ylabel('Binary Count',fontsize=14)
# plt.xscale('log')
# plt.show()

# bins = np.linspace(0,90,46)
# plt.hist(90-abs(np.degrees(catalog['inclination'])-90), bins, histtype='bar', ec='black', color='C1', density=True)
# plt.title('Marginal Inclination Distribution for Population', fontsize=15)
# plt.xlabel('Inclination (degrees)',fontsize=14)
# plt.xticks(np.linspace(0,90,10))
# #plt.ylabel('Binary Count',fontsize=14)
# plt.show()

# bins = 10**np.linspace(-1.5, 1.4, 56)
# plt.hist(catalog['distance']/1000., bins, histtype='bar', ec='black', color='C1', density=True)
# plt.title('Marginal Distance Distribution for Population', fontsize=15)
# plt.xlabel('Distance (kpc)',fontsize=14)
# #plt.ylabel('Binary Count',fontsize=14)
# plt.xscale('log')
# plt.show()

In [ ]:
# Create gbmcmc input file for selected binaries
gbmcmc_columns = ['f','fdot','colatitude','longitude','amplitude','inclination','polarization','phase']
gbmcmc = pd.DataFrame(columns = gbmcmc_columns)
for col_name in gbmcmc_columns:
    gbmcmc[col_name] = catalog[cut][col_name]
gbmcmc.to_csv('Binary_Parameters.dat',sep=' ',header=None,index=False)

In [ ]:
# Create gbfisher data file for selected binaries for comparison with gwemlisa data
output = pd.DataFrame(columns = output_columns)
for col_name in output_columns:
    output[col_name] = catalog[cut][col_name]
output.to_csv('gbfisher_parameters.dat',sep=' ',header=None,index=False)

In [ ]:
# f = plt.figure()
# bins = 10**np.linspace(0.1, 2, 29)
# plt.hist(2/(60*catalog['f']), bins, histtype='bar', label='population', ec='black', color='C1', alpha=0.5, density=True)
# plt.hist(2/(60*catalog['f'][cut]), bins, histtype='bar', label='sample', ec='black', color='C0', alpha=0.5, density=True)
# plt.title('Marginal Period Distribution', fontsize=15)
# plt.xlabel('Period (minutes)',fontsize=14)
# plt.xscale('log')
# plt.legend()
# plt.show()

# f = plt.figure()
# bins = np.linspace(0,90,46)
# plt.hist(90-abs(np.degrees(catalog['inclination'])-90), bins, histtype='bar', label='population', ec='black', color='C1', alpha=0.5, density=True)
# plt.hist(90-abs(np.degrees(catalog[cut]['inclination'])-90), bins, histtype='bar', label='sample', ec='black', color='C0', alpha=0.5, density=True)
# plt.title('Marginal Inclination Distribution', fontsize=15)
# plt.xlabel('Inclination (degrees)',fontsize=14)
# plt.xticks(np.linspace(0,90,10))
# plt.legend()
# plt.show()

# f = plt.figure()
# bins = 10**np.linspace(-1.5, 1.4, 31)
# #bins = np.linspace(0,25,26)
# plt.hist(catalog['distance']/1000., bins, histtype='bar', label='population', ec='black', color='C1', alpha=0.5, density=True)
# plt.hist(catalog['distance'][cut]/1000., bins, histtype='bar', label='sample', ec='black', color='C0', alpha=0.5, density=True)
# plt.title('Marginal Distance Distribution', fontsize=15)
# plt.xlabel('Distance (kpc)',fontsize=14)
# plt.xscale('log')
# plt.legend()
# plt.show()

In [ ]:
# f = plt.figure()
# bins = 10**np.linspace(0.7, 3.5, 33)
# plt.hist(catalog['SNR'], bins, histtype='bar', label='population', ec='black', color='C1', alpha=0.5, density=True)
# plt.hist(catalog['SNR'][cut], bins, histtype='bar', label='sample', ec='black', color='C0', alpha=0.5, density=True)
# plt.title('Signal-to-Noise Ratio Distribution', fontsize=14)
# plt.xlabel('Signal-to-Noise Ratio',fontsize=10)
# plt.ylabel('Normalized Bin Count',fontsize=10)
# plt.xscale('log')
# plt.yscale('log')
# plt.legend()
# plt.show()